<a href="https://colab.research.google.com/github/choycoy/choycoy.github.io/blob/main/lightgcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow==2.11.0

In [ ]:
!git clone https://github.com/microsoft/recommenders.git ./recommenders_microsoft

In [ ]:
import sys
sys.path.append('/content/recommenders_microsoft')
sys.path.append('/content/recommenders_microsoft/recommenders')

In [ ]:
pip install scrapbook

In [ ]:
import sys
import os
import scrapbook as sb
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

In [ ]:
# top k items to recommend
TOP_K = 10

SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = "/content/recommenders_microsoft/recommenders/models/deeprec/config/lightgcn.yaml"
user_file = "/content/recommenders_microsoft/recommenders/tests/resources/deeprec/lightgcn/user_embeddings.csv"
item_file = "/content/recommenders_microsoft/recommenders/tests/resources/deeprec/lightgcn/item_embeddings.csv"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
columns_name=['userID','itemID','elapsed_time', 'timestamp']
df = pd.read_csv("/content/drive/My Drive/final/combined_with_filename - Copy (3).csv",sep=",",names=columns_name)
print('User Dataset 1 shape : {}'.format(df.shape))
print(df)

In [ ]:
df = df.drop(df.index[0])
# Assuming df is your DataFrame
df = df.reset_index(drop=True)

In [ ]:
# Read the CSV file and convert the elapsed_time column to numeric
df['elapsed_time'] = pd.to_numeric(df['elapsed_time'])

# Calculate quartiles of elapsed time
quartiles = df['elapsed_time'].quantile([0.2, 0.4, 0.6, 0.8])

# Define rating_bins based on quartiles and add 'inf' for values greater than the 80th percentile
rating_bins = [0] + quartiles.tolist() + [float('inf')]

# Create corresponding ratings based on the number of bins (5-point rating scale)
ratings = [1, 2, 3, 4, 5]

# Create a new column for ratings
df['rating'] = pd.cut(df['elapsed_time'], bins=rating_bins, labels=ratings, include_lowest=True)

df.drop('elapsed_time', axis=1, inplace=True)
# Print the dataframe with the new 'rating' column
print(df)


In [ ]:
df['rating'] = df['rating'].astype(float)
df['timestamp'] = df['timestamp'].astype(float)

In [ ]:
train, test = python_stratified_split(df, ratio=0.75)

In [ ]:
data = ImplicitCF(train=train, test=test, seed=SEED)

In [ ]:
hparams = prepare_hparams(yaml_file,
                          n_layers=3,
                          batch_size= 4096,
                          epochs=5,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [ ]:
model = LightGCN(hparams, data, seed=SEED)

In [ ]:
with Timer() as train_time:
    model.fit()
print("Took {} seconds for training.".format(train_time.interval))

In [ ]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head()

In [ ]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')